#  Замыкание (Closure)

[Википедия](https://ru.wikipedia.org/wiki/Замыкание_(программирование))

В Python можно возвращать не только результаты вычислений, но и другие функции. Например, можно возвращать внутренние функции. Страшно? Давайте разбираться.

Давайте рассмотрим небольшой пример такого механизма.

Пусть у нас есть внешняя функция `outer()`, в которой объявлена переменная `x` и внутренняя функция `inner()`, которая просто выводит значение переменной `x` на экран. Но отличие данной конструкции от тех, что мы рассматривали ранее, состоит в том, что внешняя функция возвращает внутреннюю.

In [ ]:
#объявляем внешнюю функцию
def outer():
    x = 1 #создаём локальную переменную
    #объявляем внутреннюю функцию
    def inner():
        print('x in outer function: ', x)
    #внешняя функция возвращает внутреннюю
    return inner

Попробуем вызвать функцию `outer()` и положить результат её работы в переменную `out`. Затем выведем содержимое:

In [ ]:
out = outer()
print(out)
#Получим <function outer.<locals>.inner at 0x0000025DAA3C4700>

Итак, результат работы функции `outer()` — это функция. Попробуем вызвать и её:

In [ ]:
out()
# x in outer function:  1

Таким образом, в переменной `out` содержится ссылка на функцию `inner()`.

Теперь посмотрим на более короткую запись того, как можно «достучаться» до функции `inner()`. Действуем по принципу матрёшки: открываем большую матрёшку → вызываем внешнюю функцию → получаем внутреннюю функцию, открываем матрёшку поменьше → вызываем внутреннюю функцию:

In [ ]:
outer()()
# x in outer function:  1

x in outer function:  1


Теперь, обладая этим знанием, мы сможем справиться со следующим примером.

---

> Перед нами стоит следующая задача: подсчитать, сколько раз функция вызывается в программе. Такая задача может возникнуть, когда функция вызывается при каком-то определённом условии и вам необходимо отслеживать, сколько раз это условие выполняется. Например, у вас есть система рекомендаций музыки, и вы должны отследить, сколько раз с момента запуска программы вызвалась функция переключения музыки, чтобы понять, нравится ли пользователю представленная ему подборка.

---

Давайте в теле внешней функции `counter()` напишем внутреннюю функцию-счётчик `add()`, которая с каждым запуском будет считать, сколько раз её вызвали. 

Во внешней функции объявим переменную `number`. Во внутренней функции будем увеличивать значение этой переменной на 1. Обязательно будем использовать оператор `nonlocal`, чтобы показать, что текущее значение счётчика необходимо брать из внешней функции и оно будет изменяться. Говоря профессиональным языком, мы создаём замыкание.

In [ ]:
# Функция, которая создаёт счётчик
def counter():
    # Начальное значение счётчика — 0
    number = 0
    # Функция add будет каждый раз прибавлять
    # к счётчику 1 при запуске
    def add():
        # Сообщаем, что number берём из
        # внешней функции
        nonlocal number
        # Увеличиваем значение счётчика на 1   
        number += 1
        # Возвращаем текущее число запусков счётчика
        return number
    # Возвращаем не результат вычислений,
    # а непосредственно саму функцию add
    # без круглых скобок!
    return add

Теперь при вызове функции `counter()` будет возвращаться функция в виде объекта, который можно использовать отдельно от изначальной функции `counter()`. Эта функция, хранящаяся в виде нового объекта, будет обладать возможностями функции `add()`, которая была задана внутри внешней функции `counter()`.

Прежде чем воспользоваться новым счётчиком, его необходимо создать, вызвав функцию `counter()`, и сохранить в какую-нибудь переменную, например `counter1`:

In [ ]:
# В переменную counter1 сохраняем новый счётчик
counter1 = counter()
 
# Проверим, что counter1 действительно является вызываемым объектом,
# то есть функцией. Для этого воспользуемся встроенной функцией
# callable:
print(callable(counter1))
# Будет напечатано:
# True

Чтобы воспользоваться только что полученной функцией `counter1` для увеличения хранящегося в ней значения на `1`, достаточно написать круглые скобки после переменной `counter1`, то есть воспользоваться привычным синтаксисом вызова функции:

In [ ]:
# Запустим counter как обычную функцию
print(counter1())
print(counter1())
print(counter1())
# Будет напечатано:
# 1
# 2
# 3

1
2
3


Как видите, функция `counter1` действительно считает, сколько раз её запустили из кода. 

Давайте убедимся, что каждый счётчик считает только свои запуски, то есть не происходит глобального изменения переменной `number` во всей программе:

In [ ]:
# Создадим два различных счётчика
counter1 = counter()
counter2 = counter()

# Будем запускать вразнобой разные счётчики

print("Counter 1:", counter1())
print("Counter 1:", counter1())
print("Counter 2:", counter2())
print("Counter 1:", counter1())
print("Counter 2:", counter2())
print("Counter 2:", counter2())

# Будет напечатано:
# Counter 1: 1
# Counter 1: 2
# Counter 2: 1
# Counter 1: 3
# Counter 2: 2
# Counter 2: 3

Действительно, каждый счётчик считает только свои запуски.






### **Что здесь произошло?**

В нашем коде мы дважды вызвали функцию `counter()` и записали результаты её работы в две различные переменные: `counter1` и `counter2`. В каждой из них содержится ссылка на внутреннюю функцию add(), в которой происходит увеличение нелокальной переменной number на 1. 

Ключевым в понимании кода выше является тот факт, что для каждого из вызовов функции `counter()` переменная number является локальной, то есть для каждого вызова она своя. Переменная number создаётся при каждом вызове внешней функции `counter()`, а мы вызвали её дважды.

Таким образом, когда мы вызвали функцию `counter()` в первый раз, создалась первая переменная number:

`counter1 = counter()`

Когда мы вызвали функцию `counter()` во второй раз, создалась вторая переменная number:

`counter2 = counter()`

Однако она не имеет никакого отношения к первой. Изменение первой переменной не влечёт за собой изменение второй, и наоборот.

Поэтому, когда мы вызываем функцию `add()` от имени переменных `counter1` и `counter2`, мы изменяем значения двух различных переменных `number`. 

Получается, что для каждой функции ведётся индивидуальный подсчёт её вызовов.

> В этом и заключается преимущество использования замыканий по сравнению с модификацией глобальных переменных в теле функции: мы увеличиваем значение переменной только в рамках одной функции.

### **Когда это может вам понадобиться?**


На самом деле замыкания — это инструмент, который позволяет избежать использования глобальных переменных, и обеспечивает некоторую форму сокрытия переменных: мы обеспечиваем работу двух одинаковых интерфейсов (функций) независимо друг от друга. К тому же замыкания — это первый шаг к пониманию более сложных конструкций, которые мы изучим далее — **декораторов**.

Представим ситуацию

---

Для построения модели, которая будет предсказывать поведение пользователей в приложении (например, сделает клиент заказ или нет), нам может пригодиться информация о том, сколько раз каждый клиент кликал на кнопку «Купить». 

Предположим, у нас уже есть функция, которая привязана к этой кнопке, то есть она вызывается при нажатии кнопки покупки. Нам необходимо постоянно вести подсчёт, сколько раз вызывается эта функция, но не в глобальном смысле (не для всех пользователей в общем), а для каждого пользователя независимо от других. Тут нам и могут пригодиться замыкания. Мы можем организовать внутреннюю функцию-замыкание, в которой будем увеличивать количество нажатий кнопки «Купить».

---


В этом юните вы узнали о порядке разрешения переменных в Python и научились:

*   создавать вложенные функции;
*   использовать нелокальные и глобальные переменные в коде функции;
*   возвращать функции вместо результатов их выполнения и использовать их в основном скрипте.
